## Chatbot interactif Phi 3 Mini 4K Instruct avec Whisper

### Introduction :
Le chatbot interactif Phi 3 Mini 4K Instruct est un outil qui permet aux utilisateurs d'interagir avec la démonstration Microsoft Phi 3 Mini 4K Instruct en utilisant une entrée textuelle ou audio. Le chatbot peut être utilisé pour diverses tâches, telles que la traduction, les mises à jour météo et la collecte d'informations générales.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Créer votre jeton d'accès Huggingface](https://huggingface.co/settings/tokens)

Créer un nouveau jeton  
Donner un nouveau nom  
Sélectionner les permissions d'écriture  
Copier le jeton et le conserver dans un endroit sûr


Le code Python suivant effectue deux tâches principales : importer le module `os` et définir une variable d'environnement.

1. Importation du module `os` :
   - Le module `os` en Python offre un moyen d'interagir avec le système d'exploitation. Il permet d'effectuer diverses tâches liées au système, comme accéder aux variables d'environnement, manipuler des fichiers et des répertoires, etc.
   - Dans ce code, le module `os` est importé à l'aide de l'instruction `import`. Cette instruction rend les fonctionnalités du module `os` disponibles pour une utilisation dans le script Python actuel.

2. Définition d'une variable d'environnement :
   - Une variable d'environnement est une valeur accessible par les programmes exécutés sur le système d'exploitation. Elle permet de stocker des paramètres de configuration ou d'autres informations pouvant être utilisées par plusieurs programmes.
   - Dans ce code, une nouvelle variable d'environnement est définie en utilisant le dictionnaire `os.environ`. La clé du dictionnaire est `'HF_TOKEN'`, et la valeur est assignée à partir de la variable `HUGGINGFACE_TOKEN`.
   - La variable `HUGGINGFACE_TOKEN` est définie juste au-dessus de cet extrait de code, et elle reçoit une valeur de type chaîne de caractères `"hf_**************"` à l'aide de la syntaxe `#@param`. Cette syntaxe est souvent utilisée dans les notebooks Jupyter pour permettre la saisie utilisateur et la configuration des paramètres directement dans l'interface du notebook.
   - En définissant la variable d'environnement `'HF_TOKEN'`, celle-ci peut être accessible par d'autres parties du programme ou par d'autres programmes exécutés sur le même système d'exploitation.

En résumé, ce code importe le module `os` et définit une variable d'environnement nommée `'HF_TOKEN'` avec la valeur fournie dans la variable `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ce snippet de code définit une fonction appelée clear_output, utilisée pour effacer le contenu affiché dans la cellule actuelle d'un Jupyter Notebook ou d'IPython. Décortiquons le code et comprenons son fonctionnement :

La fonction clear_output prend un paramètre appelé wait, qui est une valeur booléenne. Par défaut, wait est défini sur False. Ce paramètre détermine si la fonction doit attendre qu'une nouvelle sortie soit disponible pour remplacer l'ancienne avant de l'effacer.

La fonction elle-même est utilisée pour effacer le contenu affiché dans la cellule actuelle. Dans un Jupyter Notebook ou IPython, lorsqu'une cellule produit une sortie, comme du texte imprimé ou des graphiques, cette sortie est affichée sous la cellule. La fonction clear_output permet de supprimer cette sortie.

L'implémentation de la fonction n'est pas fournie dans le snippet de code, comme indiqué par les points de suspension (...). Ces points de suspension représentent un espace réservé pour le code réel qui effectue l'effacement de la sortie. L'implémentation de la fonction pourrait impliquer une interaction avec l'API de Jupyter Notebook ou d'IPython pour supprimer la sortie existante de la cellule.

En résumé, cette fonction offre un moyen pratique d'effacer le contenu affiché dans la cellule actuelle d'un Jupyter Notebook ou d'IPython, facilitant ainsi la gestion et la mise à jour des sorties affichées lors de sessions de codage interactives.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Effectuer la synthèse vocale (TTS) en utilisant le service Edge TTS. Passons en revue les implémentations des fonctions pertinentes une par une :

1. `calculate_rate_string(input_value)` : Cette fonction prend une valeur en entrée et calcule la chaîne de taux pour la voix TTS. La valeur d'entrée représente la vitesse souhaitée de la parole, où une valeur de 1 représente la vitesse normale. La fonction calcule la chaîne de taux en soustrayant 1 de la valeur d'entrée, en la multipliant par 100, puis en déterminant le signe en fonction de si la valeur d'entrée est supérieure ou égale à 1. La fonction retourne la chaîne de taux au format "{signe}{taux}".

2. `make_chunks(input_text, language)` : Cette fonction prend un texte en entrée et une langue comme paramètres. Elle divise le texte en morceaux en fonction des règles spécifiques à la langue. Dans cette implémentation, si la langue est "English", la fonction divise le texte à chaque point (".") et supprime les espaces en début ou en fin de chaîne. Elle ajoute ensuite un point à chaque morceau et retourne la liste filtrée des morceaux.

3. `tts_file_name(text)` : Cette fonction génère un nom de fichier pour le fichier audio TTS basé sur le texte en entrée. Elle effectue plusieurs transformations sur le texte : suppression d'un point final (si présent), conversion du texte en minuscules, suppression des espaces en début et en fin de chaîne, et remplacement des espaces par des underscores. Elle tronque ensuite le texte à un maximum de 25 caractères (si plus long) ou utilise le texte complet s'il est vide. Enfin, elle génère une chaîne aléatoire en utilisant le module [`uuid`] et la combine avec le texte tronqué pour créer le nom de fichier au format "/content/edge_tts_voice/{texte_tronqué}_{chaîne_aléatoire}.mp3".

4. `merge_audio_files(audio_paths, output_path)` : Cette fonction fusionne plusieurs fichiers audio en un seul fichier audio. Elle prend une liste de chemins de fichiers audio et un chemin de sortie comme paramètres. La fonction initialise un objet `AudioSegment` vide appelé [`merged_audio`]. Elle parcourt ensuite chaque chemin de fichier audio, charge le fichier audio en utilisant la méthode `AudioSegment.from_file()` de la bibliothèque `pydub`, et ajoute le fichier audio actuel à l'objet [`merged_audio`]. Enfin, elle exporte l'audio fusionné vers le chemin de sortie spécifié au format MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)` : Cette fonction effectue l'opération TTS en utilisant le service Edge TTS. Elle prend une liste de morceaux de texte, la vitesse de la parole, le nom de la voix et le chemin de sauvegarde comme paramètres. Si le nombre de morceaux est supérieur à 1, la fonction crée un répertoire pour stocker les fichiers audio des morceaux individuels. Elle parcourt ensuite chaque morceau, construit une commande Edge TTS en utilisant la fonction `calculate_rate_string()`, le nom de la voix et le texte du morceau, et exécute la commande en utilisant la fonction `os.system()`. Si l'exécution de la commande réussit, elle ajoute le chemin du fichier audio généré à une liste. Après avoir traité tous les morceaux, elle fusionne les fichiers audio individuels en utilisant la fonction `merge_audio_files()` et sauvegarde l'audio fusionné au chemin de sauvegarde spécifié. S'il n'y a qu'un seul morceau, elle génère directement la commande Edge TTS et sauvegarde l'audio au chemin de sauvegarde. Enfin, elle retourne le chemin de sauvegarde du fichier audio généré.

6. `random_audio_name_generate()` : Cette fonction génère un nom de fichier audio aléatoire en utilisant le module [`uuid`]. Elle génère un UUID aléatoire, le convertit en chaîne, prend les huit premiers caractères, ajoute l'extension ".mp3", et retourne le nom de fichier audio aléatoire.

7. `talk(input_text)` : Cette fonction est le point d'entrée principal pour effectuer l'opération TTS. Elle prend un texte en entrée comme paramètre. Elle vérifie d'abord la longueur du texte en entrée pour déterminer s'il s'agit d'une phrase longue (600 caractères ou plus). En fonction de la longueur et de la valeur de la variable `translate_text_flag`, elle détermine la langue et génère la liste des morceaux de texte en utilisant la fonction `make_chunks()`. Elle génère ensuite un chemin de sauvegarde pour le fichier audio en utilisant la fonction `random_audio_name_generate()`. Enfin, elle appelle la fonction `edge_free_tts()` pour effectuer l'opération TTS et retourne le chemin de sauvegarde du fichier audio généré.

Dans l'ensemble, ces fonctions travaillent ensemble pour diviser le texte en morceaux, générer un nom de fichier pour le fichier audio, effectuer l'opération TTS en utilisant le service Edge TTS, et fusionner les fichiers audio individuels en un seul fichier audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

La mise en œuvre de deux fonctions : convert_to_text et run_text_prompt, ainsi que la déclaration de deux classes : str et Audio.

La fonction convert_to_text prend un chemin audio (audio_path) en entrée et transcrit l'audio en texte à l'aide d'un modèle appelé whisper_model. La fonction commence par vérifier si le drapeau gpu est défini sur True. Si c'est le cas, le whisper_model est utilisé avec certains paramètres tels que word_timestamps=True, fp16=True, language='English', et task='translate'. Si le drapeau gpu est False, le whisper_model est utilisé avec fp16=False. La transcription obtenue est ensuite sauvegardée dans un fichier nommé 'scan.txt' et retournée sous forme de texte.

La fonction run_text_prompt prend un message et un historique de conversation (chat_history) en entrée. Elle utilise la fonction phi_demo pour générer une réponse d'un chatbot basée sur le message d'entrée. La réponse générée est ensuite passée à la fonction talk, qui convertit la réponse en un fichier audio et retourne le chemin du fichier. La classe Audio est utilisée pour afficher et lire le fichier audio. L'audio est affiché à l'aide de la fonction display du module IPython.display, et l'objet Audio est créé avec le paramètre autoplay=True, ce qui permet à l'audio de démarrer automatiquement. L'historique de conversation est mis à jour avec le message d'entrée et la réponse générée, et une chaîne vide ainsi que l'historique de conversation mis à jour sont retournés.

La classe str est une classe intégrée de Python qui représente une séquence de caractères. Elle fournit diverses méthodes pour manipuler et travailler avec des chaînes de caractères, telles que capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, et bien d'autres. Ces méthodes permettent d'effectuer des opérations comme la recherche, le remplacement, le formatage et la manipulation de chaînes de caractères.

La classe Audio est une classe personnalisée qui représente un objet audio. Elle est utilisée pour créer un lecteur audio dans l'environnement Jupyter Notebook. La classe accepte divers paramètres tels que data, filename, url, embed, rate, autoplay, et normalize. Le paramètre data peut être un tableau numpy, une liste d'échantillons, une chaîne représentant un nom de fichier ou une URL, ou des données PCM brutes. Le paramètre filename est utilisé pour spécifier un fichier local à partir duquel charger les données audio, et le paramètre url est utilisé pour spécifier une URL à partir de laquelle télécharger les données audio. Le paramètre embed détermine si les données audio doivent être intégrées à l'aide d'un URI de données ou référencées à partir de la source originale. Le paramètre rate spécifie la fréquence d'échantillonnage des données audio. Le paramètre autoplay détermine si l'audio doit commencer à jouer automatiquement. Le paramètre normalize spécifie si les données audio doivent être normalisées (recalibrées) à la plage maximale possible. La classe Audio fournit également des méthodes comme reload pour recharger les données audio à partir d'un fichier ou d'une URL, et des attributs comme src_attr, autoplay_attr, et element_id_attr pour récupérer les attributs correspondants pour l'élément audio en HTML.

En résumé, ces fonctions et classes sont utilisées pour transcrire l'audio en texte, générer des réponses audio à partir d'un chatbot, et afficher et lire des fichiers audio dans l'environnement Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
